#### Dependencies
___

In [224]:
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession

#### Create Context
____

In [266]:
conf = SparkConf().setMaster('local[2]').setAppName('ApacheWebLogsStream')
sc = SparkContext(conf=conf)

#### Create Streaming Context
____

In [267]:
ssc = StreamingContext(sparkContext=sc, batchDuration=10) 
input_stream = ssc.socketTextStream(hostname='localhost', port=9999)

In [268]:
def process(time, rdd):
    print(time)
    
    spark = SparkSession.builder.appName('processLR').getOrCreate()
    
    rows = []
    header = ['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership', 'Yearly Amount Spent']
    
    for row in rdd.map(lambda line: line.split(",")).collect():
        rows.append((*row,))
    
    if (len(rows)) > 0:
        dataFrame = spark.createDataFrame(rows, header,)
    
    #rows = rdd.map(lambda line: line.split(",")).collect()
    #rowRdd = rdd.map(lambda w: Row(word=w))
    #dataFrame = spark.createDataFrame(rowRdd)
    
    #rowRDD = rdd.map(lambda line: line.split(','))
    #dataFrame = rowRDD.toDF(['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership', 'Yearly Amount Spent'])  
    
        dataFrame.printSchema()
        dataFrame.show(n=5)    
    
    print('done')
    
    

In [269]:
#words = lines.flatMap(lambda line: line.split(' '))
#pairs = words.map(lambda word: (word, 1))
#word_counts = pairs.reduceByKey(lambda num1, num2: num1+num2)
#word_counts.pprint()

#print(type(lines))
#print(dir(lines))
#input_stream = lines.flatMap(lambda line: line.split('\n'))
#input_stream.foreachRDD(process)

input_stream.foreachRDD(process)


#### Start Streaming
____

In [270]:
ssc.start()

2020-05-03 14:08:50
done
2020-05-03 14:09:00
done
2020-05-03 14:09:10
root
 |-- Avg Session Length: string (nullable = true)
 |-- Time on App: string (nullable = true)
 |-- Time on Website: string (nullable = true)
 |-- Length of Membership: string (nullable = true)
 |-- Yearly Amount Spent: string (nullable = true)

+------------------+-----------+---------------+--------------------+-------------------+
|Avg Session Length|Time on App|Time on Website|Length of Membership|Yearly Amount Spent|
+------------------+-----------+---------------+--------------------+-------------------+
|       34.49726773|12.65565115|    39.57766802|         4.082620633|         587.951054|
|       31.92627203|11.10946073|    37.26895887|         2.664034182|        392.2049334|
|       33.00091476|11.33027806|    37.11059744|         4.104543202|        487.5475049|
|       34.30555663|13.71751367|    36.72128268|         3.120178783|         581.852344|
|       33.33067252|12.79518855|     37.5366533|   

In [272]:
ssc.awaitTermination()

ssc.stop()